In [ ]:
!pip install -U langchain_openai
!pip install -U langchain
!pip install -U langchain_community
!pip install -U unstructured
!pip install -U pdfminer.six
!pip install pi-heif
!pip install unstructured-inference
!pip install chromadb --use-deprecated=legacy-resolver

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import json
import os

In [ ]:
## 1번째 단계: 데이터 로드

from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader('./2024_Annual_Report')
report_2024_data = loader.load()    # 디렉토리 내의 pdf 가져와서 "페이지별로" 문서 객체로 변환

len(report_2024_data)             # 따라서 페이지 수대로 나옴
# 출력: 187

In [ ]:
report_2024_data[0]
# 출력: Document(metadata={'source': '2024_Annual_Report/NVIDIA-2024-Annual-Report.pdf', 'page': 0}, page_content='2024  \nNVIDIA Corporation\nAnnual Review \n \nNotice of Annual Meeting \nProxy Statement \nForm 10-K')

In [ ]:
## 2번째 단계: Text split (텍스트 분할)

import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

def protect_decimal_points(text): # 소수점 "." 보호
    return re.sub(r'(\d)\.(\d)', r'\1<SPLIT_PROTECTED>\2', text)  # 정수.정수 패턴에서 "."을 임시로 보호 문자로 교체

def restore_decimal_points(text): # 보호된 "."을 복원
    return text.replace('<SPLIT_PROTECTED>', '.')

text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        ".",
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "\n",
        " ",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "",
    ],
    chunk_size=200,       # 텍스트를 분할할 때 분할된 청크들이 설정된 chunk_size보다 작아질 때까지 과정을 반복
    chunk_overlap=100,    # 분할된 텍스트 조각들 사이에서 중복으로 포함될 문자 수
    length_function=len,
)

protected_text = protect_decimal_points(report_2024_data[2].page_content) # 소수점 먼저 보호

texts = text_splitter.split_text(protected_text)

final_texts = [restore_decimal_points(text) for text in texts]            # 보호된 소수점 복원

for i, chunk in enumerate(final_texts):
    print(f"Chunk {i+1}: {chunk}")

# 출력: 
# ['“The sum of all that \nNVIDIA’s doing \nwill indeed create \nthe next industrial \nrevolution”\nCNBC\nAccelerated computing is sustainable \ncomputing',
#  '. Every data center in the world \nneeds to be accelerated to reclaim power, \nachieve sustainability, and realize net-zero \nemissions',
#  '. Accelerated data centers could save \nan incredible 19 terawatt-hours of electricity \nannually if run on GPU and DPU accelerators vs \nCPUs',
#  '. That’s about the same energy as a year’s \nworth of trips by 2.9 million passenger cars. \nThe efficiency of accelerated computing \npaved the way for generative AI',
#  '. The most \ncritical computing platform of our generation, \ngenerative AI will reshape the world’s largest \nindustries and create an entirely new one',
#  '.\nNVIDIA, the pioneer of accelerated computing, \nis the driving force of this new era.']

In [ ]:
# pdf 전체에 대해서 텍스트 분할
for i, page in enumerate(report_2024_data):
    protected_text = protect_decimal_points(page.page_content)
    texts = text_splitter.split_text(protected_text)
    SinglePage_texts = [restore_decimal_points(text) for text in texts]
    print(f"Page {i+1}: {SinglePage_texts}")   # SinglePage_texts : 페이지 단위로 있는 청크 묶음 List -> 총 187개 있음
    print()

    # "Page {i+1}: {SinglePage_texts}"를 report2024.txt로 저장
    with open("./2024_Annual_Report/report2024.txt", "a", encoding="utf-8") as f:
        f.write(f"Page {i+1}: {SinglePage_texts}\n")

# 출력:
# Page 1: ['2021\nNVIDIA CORPORATION \nANNUAL REVIEW\nNOTICE OF ANNUAL MEETING\nPROXY STATEMENT \nFORM 10-K']

# Page 2: ['“AMERICA’S \nMOST IMPORTANT \nCOMPANY”\nFORBES\nThis was a year that none of us will forget. \nBut extraordinary companies rise in extraordinary \ntimes', '. We are building a once-in-a-generation \ncompany to tackle the world’s greatest \nchallenges—one that made these extraordinary \ntimes into one of our finest hours.\nJensen Huang']

# Page 3: []

# Page 4: ['“WE’RE SEEING A MAJOR \nCHANGE IN HOW WE THINK OF \nPC GRAPHICS, AND IT SHOULDN’T \nBE A SURPRISE THAT NVIDIA IS \nLEADING THE WAY”\nTECH RADAR\nGaming is the world’s largest entertainment \nindustry', '. With more than 200 million gamers, \nNVIDIA GeForce is its largest platform. GeForce \nRTX GPUs and the GeForce Experience \napplication transform everyday PCs into \npowerful gaming machines', '.\nPowered by the NVIDIA Ampere architecture—\nour 2nd-gen RTX design, GeForce RTX 30 Series \nGPUs deliver incredibly realistic ray-traced \ngraphics and cutting-edge AI features for \ngamers', '. And with Max-Q design, 30 Series \nlaptops can deliver twice the power efficiency \nof previous generations.']

# Page 5: []

# Page 6: ['“THE NEXT FRONTIER \nFOR REAL -TIME \nCOLLABORATION FOR \n3D CONTENT CREATION”\nTWITTER TECH NEWS\nNVIDIA Omniverse is an open platform for \ncollaboration and real-time photorealistic \nsimulation', '. This virtual world brings together \ncomplex workflows of digital creation, design, \nand engineering to provide physically accurate \nresults in 3D', '. More than a design environment, \nOmniverse is a metaverse where creators can \nconvene, worlds can be created, and AI agents \nlike robots and autonomous vehicles can train.']

# Page 7: ['Made in NVIDIA Omniverse']

# Page 8: ["“IT'S OFFICIAL: \nNVIDIA IS NOT JUST \nA GAMING COMPANY \nANYMORE”\nDATA CENTER KNOWLEDGE\nWe used to think of a CPU server as the basic unit \nof computing", '. But to meet the demands of today’s \nmachine learning and AI workloads, we must \noptimize the entire data center, from end to end', '. \nIt’s an orchestration of three pillars: the CPU for \ngeneral-purpose computing, the GPU for \naccelerated computing, and the DPU, which \nprocesses and moves data around the data center', '. \nNVIDIA DGX SuperPOD brings them together to \ncreate the world’s first turnkey AI infrastructure—a \nrocket engine to power the entire machine learning', 'rocket engine to power the entire machine learning \npipeline, from data loading and processing, to \ntraining, to inference', '.\nFACE ALIGNMENTAI COMPRESSION TRANSLATION']

# Page 9: ['Made in NVIDIA Omniverse\nNVIDIA Maxine is a cloud-AI video streaming platform that can breathe new life into the video calls that bring us \ntogether for work, study, and personal connection', '. The latest platform brought to life by NVIDIA AI, it joins NVIDIA \nJarvis for conversational AI and NVIDIA Merlin for large-scale recommender systems that predict user', 'preferences like products a shopper would buy, movies to watch, or news of interest', '.']

# Page 10: ["“MERCEDES-BENZ AND NVIDIA \nSIGN A DEAL TO MAKE CARS \nMORE LIKE PHONES”\nBARRON'S\nTomorrow’s cars and trucks will be high-performance, \nupdatable computing devices", '. Leading the way, Mercedes-\nBenz is working with NVIDIA to bring software-defined \nvehicles to its fleet starting in 2024', '. The next-generation \nMercedes vehicles will be powered by a first-of-its-kind \ncomputing architecture based on the NVIDIA DRIVE \nplatform and backed by a team of AI engineers and', 'platform and backed by a team of AI engineers and \npowerful AI supercomputers', '.\nDATA COLLECTING,\nTESTING\nDRIVE\nHyperion\nDGX\nA100\nDRIVE\nConstellation\nAV on\nDRIVE AGX\nIX on\nDRIVE AGX\nMAPPING, TRAINING SIMULATION AV DRIVING AV WORLD MODEL']

# Page 11: ['Made in NVIDIA Omniverse\nEssentially data centers on wheels, autonomous vehicles may be the greatest machine learning and robotics \nchallenge today. Their economic and societal impact will be huge', '. Their economic and societal impact will be huge. NVIDIA DRIVE is an end-to-end platform to \ndevelop the software-defined autonomous vehicles of tomorrow', '. It brings together processors, computers, sensor \narchitecture, data processing, mapping, driving software, simulation, fleet management, and road testing—all of it', 'to the highest functional safety and cybersecurity standards', '. Customers and partners in the transportation \nindustry can use all or parts of the DRIVE platform to develop autonomous vehicles at scale.']

# Page 12: ['“WITH NVIDIA EGX, HOSPITALS,\nSTORES, AND FACTORIES CAN\nCARRY OUT REAL -TIME PROCESSING \nOF MASSIVE AMOUNTS OF DATA\nFROM TRILLIONS OF SENSORS”\nINSIDE HPC', 'CARRY OUT REAL -TIME PROCESSING \nOF MASSIVE AMOUNTS OF DATA\nFROM TRILLIONS OF SENSORS”\nINSIDE HPC\nAI is expanding outward from the cloud and into the edge, where', 'INSIDE HPC\nAI is expanding outward from the cloud and into the edge, where \ntrillions of devices, sensors, and autonomous machines are \ngenerating large pools of data', '. Sensors connected to AI \ncomputers can make seemingly intelligent decisions to speed \ncheckouts, direct forklifts, manage traffic, and save power', '. As 5G, \nAI, and autonomous machines converge, almost everything \naround us can be smart, cloud-connected, and based on \nplatforms that can deliver countless new services', '.\nThe NVIDIA EGX Edge AI platform is bringing AI to the world’s \nlargest industries. Walmart, the US Postal Service, Procter & \nGamble, and Samsung Electronics are among the first customers.']

# Page 13: ['Jarvis\nNVIDIA \nBlueField-2\nNVIDIA Aerial \nA100\nNVIDIA \nA40/A10\nNVIDIA \nA100/A30\nMaxine Merlin DRIVEClaraMetropolisIsaacMorpheus\nNVIDIA AI ENTERPRISE\nNVIDIA GPU CLOUD REGISTRY', 'Maxine Merlin DRIVEClaraMetropolisIsaacMorpheus\nNVIDIA AI ENTERPRISE\nNVIDIA GPU CLOUD REGISTRY\nNVIDIA CUDA, DOCA, vGPU, Magnum IO, Aerial 5G, Morpheus\nVMware vSphere', 'NVIDIA GPU CLOUD REGISTRY\nNVIDIA CUDA, DOCA, vGPU, Magnum IO, Aerial 5G, Morpheus\nVMware vSphere\ncuIO cuDF cuDNN cuML cuGraph TAO TensorRT Triton\nMade in NVIDIA Omniverse']

# Page 14: ['None of us can forget this year—the year \nCOVID-19 changed the world. A full-out \nglobal pandemic, the world is at war with an \ninvisible enemy. The facts and figures of the \npandemic are staggering', '. The facts and figures of the \npandemic are staggering. The loss of \nmillions is unimaginable. Yet, it is the loss of \none among our families that makes \nCOVID-19 so real', '. Yet, it is the loss of \none among our families that makes \nCOVID-19 so real.\nThe pandemic tested the character of people, \nnations, and companies and shined a \nspotlight on what we truly value', '.\nOur core purpose is to impact by pioneering \ncomputing that can solve problems ordinary \ncomputers cannot and be a platform for \nscientists, developers, partners, industries,', 'computers cannot and be a platform for \nscientists, developers, partners, industries, \nand employees to create a better future', '.\nWe want to build a one-of-a-kind company \nthat invents the future, builds amazing \ntechnologies, and strives to achieve the \nhighest level of craft', ".\nWe prioritize our employees and create a \nplace where the world's best come to do \ntheir life's work and build a great life for \ntheir families", '.\nAt NVIDIA, the spirit of invention, the drive \nfor perfection, and a compassionate culture \nthat cares for our families and communities \nare harmonious', '.\nTAKING CARE OF FAMILY \nDURING COVID\nWe confronted the reality of the pandemic \nearly and mobilized quickly. We directed \nemployees to stay home and stay safe and \nreduce spread', '. We directed \nemployees to stay home and stay safe and \nreduce spread. People were scared and \nuncertain', ". We assured employees no one's \njob would be affected and even pulled in \nraises by six months, giving them confidence \nand a little cushion to ride out the pandemic", '. \nWe continued to pay the facility and cafeteria \ncontractors who take such great care of us', '.\nWe first kept our employees safe, and then \nwe tackled the massive undertakings to \nkeep the company running while working \nfrom home', '. We mobilized efforts worldwide \nto figure out how to engineer and launch a \nrecord number of new products, collaborate \nwith a record number of new partners in', 'record number of new products, collaborate \nwith a record number of new partners in \nnew markets, and ramp the most complex \nproducts the world has ever made—Ampere, \nBlueField, RTX, DGX', '. Our employees rose to \nthe occasion, worked around the clock, and \ninvented new and even better ways to do \nour work.\nUSING OUR SUPERPOWERS\nThen we set sights on the virus', '. Joining with \nnational labs, research hospitals, universi -\nties, startups, and pharma companies, \nNVIDIA researchers and engineers, armed \nwith our unique technologies, our superpow-', 'NVIDIA researchers and engineers, armed \nwith our unique technologies, our superpow-\ners—image processing, high-speed \nsimulation, and AI expertise—looked for', 'ers—image processing, high-speed \nsimulation, and AI expertise—looked for \nways to detect, treat, and, at the speed of \nlight, find a vaccine for the virus', '.\nLet me highlight just a few of the hundreds \nof initiatives', ".\nWe partnered with 20 hospitals from eight \ncountries and developed an AI model that \npredicts COVID patients' need for supple -\nmental oxygen in the next 24 hours", '. Using \nNVIDIA Clara federated learning, we trained \nthe AI model on data from all of the hospitals \nwithout sharing patient-private data', '.\nResearchers at UC San Diego did the largest, \nlongest, and most accurate simulation of a \ncoronavirus', '. Simulating 305 million atoms \non the 28,000-GPU Summit supercomputer, \nthe team created a virtual microscope and \nsaw how COVID-19 infects human cells', '. They \ndid in a few months what previously took five \nyears with the flu virus. Their work was \ncritical to vaccine design', '. Their work was \ncritical to vaccine design.\nOver a thousand of the almost 8,000 AI \nstartups we work with are using AI to \nrevolutionize healthcare', '. They’re doing \namazing work across the entire spectrum', '. \nWe partnered with many to counter \nCOVID-19—from the highest-ever resolution \ncryo-EM imaging, inferring the 3D atomic \nstructure of COVID-19 from 2D cryo-EM', 'cryo-EM imaging, inferring the 3D atomic \nstructure of COVID-19 from 2D cryo-EM \nimages, using AI natural language under-\nstanding to read the amino acids of COVID-19', "images, using AI natural language under-\nstanding to read the amino acids of COVID-19 \nand predict the protein's 3D structure, doing \nhigh-throughput gene sequencing to", "and predict the protein's 3D structure, doing \nhigh-throughput gene sequencing to \naccurately detect infection, simulating the \nphysical behavior of COVID-19, using AI to", 'accurately detect infection, simulating the \nphysical behavior of COVID-19, using AI to \ngenerate potential drug candidates, fusing \nphysics simulation and AI to do virtual', 'generate potential drug candidates, fusing \nphysics simulation and AI to do virtual \nscreening, to using AI to accelerate lab \nscreening by automating the analysis of cell', 'screening, to using AI to accelerate lab \nscreening by automating the analysis of cell \nimages so millions of experiments can be \nrun each week', '.\nDEAR NVIDIANS \nAND STAKEHOLDERS,\n$10\n$11\n$12\n$13\n$14\n$15\n$16\n$17\nFY21FY20\nin Billions\nGAAP results\nGROSS MARGINS\n+30 bps\n$10.92\nEPS\n+53%\nREVENUE\n+53%\n61.0%', 'GROSS MARGINS\n+30 bps\n$10.92\nEPS\n+53%\nREVENUE\n+53%\n61.0%\n61.5%\n62.0%\n62.5%\nFY21FY20\n$4.00', '61.5%\n62.0%\n62.5%\nFY21FY20\n$4.00\n$4.50\n$5.00\n$5.50\n$6.00', '$4.50\n$5.00\n$5.50\n$6.00\n$6.50\n$7.00\nFY21FY20\n$16.68\n62.3%', '$6.50\n$7.00\nFY21FY20\n$16.68\n62.3%\n62.0%\n$4.52\n$6.90\n$3.50']

# Page 15: ["The pandemic was a tragedy, but we didn't \nlet it go to waste", ". The world's scientific and \nhealthcare community is in a race against \ntime, armed with the latest technologies to \ndo in months what previously took years", '. \nThese new accelerated methods will \nprofoundly impact the pace of drug discov-\nery for all diseases. We may have advanced \nscience a decade in the past year', '. We may have advanced \nscience a decade in the past year.\nCOVID-19 will not be the last killer virus, but \nwill hopefully be the last pandemic. Next \ntime, the world will be better prepared', '. \nNVIDIA GPU computers and NVIDIA Clara, \nwith our suite of imaging, atomic and \nquantum physics simulations, and genome-\nreading AI libraries, is our computational', 'quantum physics simulations, and genome-\nreading AI libraries, is our computational \ndefense system that lets the world respond \nat the speed of light', '.\nOur work this year is set in the context of the \npowerful forces shaping the computer \nindustry. These forces, which will influence \nevery industry in the future, will shape  \nthe world', '.\nACCELERATED COMPUTING IS THE \nPATH FORWARD\nNVIDIA pioneered accelerated computing, \nwhich supercharges computationally \nintensive applications like computer \ngraphics, scientific computing, AI, and', 'intensive applications like computer \ngraphics, scientific computing, AI, and \nrobotics', '. We accomplish this with a special \nprocessor and full-stack computer science \nthat reinvents chips and systems, system \nsoftware, and applications', '. Our work spans \nthe largest cloud data centers to tiny \nautonomous machines', '.\nOur dedication to accelerated computing has \nbuilt a community of more than 2.5 million \nresearchers and developers, nearly 8,000 AI \nstartups, and all major IT platform leaders', '. \nDevelopers have downloaded CUDA and \nNVIDIA AI software over 24 million times', ". \nWhat developers do on NVIDIA is extraordi -\nnary, tackling society's greatest challenges \nand inventing a future that is impossible \notherwise. It gives us profound joy to \nsupercharge their work", '. It gives us profound joy to \nsupercharge their work.\nRTX RESETS COMPUTER GRAPHICS\nComputer graphics is the driving force of our \ncompany', '. At its core, computer graphics is \nabout simulations—using mathematics and \ncomputer science to simulate the interaction \nof light and materials; the physics of objects,', 'computer science to simulate the interaction \nof light and materials; the physics of objects, \nparticles, and waves; and now simulating \nintelligence and animation', ".\nThe science, engineering, and artistry that \nwe dedicate to simulate mother nature's \nphysics have led to incredible advances", ".\nIn simplistic terms, NVIDIA GPU \nprocessing has increased 100,000-fold  \nin 25 years and has allowed us to advance \nthe basic sciences, the arts, and the \nworld's industries", '.\nLast year, we introduced the second \ngeneration of NVIDIA RTX—a new rendering \napproach that fuses rasterization and \nprogrammable shading with hardware-\naccelerated ray tracing and AI', '. The \nculmination of 10 years of research, RTX has \nreset computer graphics. Real-time ray \ntracing has arrived.\nRTX is a home run—our most successful \nGPU generation ever', '.\nRTX is a home run—our most successful \nGPU generation ever. Demand for the \nGeForce 30 Series is incredible, driving our \ngaming business to grow over 40 percent \nthis year', ".\nWe are at the outset of the biggest upgrade \ncycle of our user base since we invented the \nGPU two decades ago. And it's just getting \nstarted", ". And it's just getting \nstarted. More than 85 percent of our \nGeForce installed base has yet to upgrade \nto RTX", '. More than 85 percent of our \nGeForce installed base has yet to upgrade \nto RTX.\nTHE NEW UNIT OF COMPUTING\nWe completed the acquisition of Mellanox \nand combined our companies during the \nshutdown', '. The teams have melded \nincredibly well. The combination of NVIDIA \nand Mellanox has expanded the scope of our \nvision and positioned us to help rearchitect \nthe data center', '. \nFor decades, we considered the CPU the \nbasic unit of computing. The rise of cloud \ncomputing and AI services is driving \nfundamental changes in computer \narchitecture', '. Hyperscale applications are \ncomposed of microservices and run across \nthe entire data center as if it’s one \ncomputer—the data center is a new unit \nof computing', '.\nData center-scale applications require \ncomputer architects to optimize across all \nthree fundamental processors: the CPU for \ngeneral-purpose computing, the GPU for', 'three fundamental processors: the CPU for \ngeneral-purpose computing, the GPU for \naccelerated computing, and the DPU, or data \nprocessing unit', '. An evolution of the \nnetworking chip, the DPU is programmable \nand processes the rapidly growing body of \nsoftware that runs in data center \ninfrastructure, such as virtualization,', 'software that runs in data center \ninfrastructure, such as virtualization, \nnetworking, storage, and security', '.\nNVIDIA pioneered GPU computing. Building \non Mellanox’s world-class networking chip, \nwe are pioneering the DPU', '. Building \non Mellanox’s world-class networking chip, \nwe are pioneering the DPU. An estimated \none-third of the CPUs in a data center are \nprocessing infrastructure software', '. \nWith NVIDIA BlueField DPUs, we can offload \nand accelerate the infrastructure, increase \nperformance and security, and free up a \nthird of the CPUs for applications', ".\nDuring the year, we announced the Blue-\nField-2 DPU, our data center infrastructure \non a chip. It's a breakthrough in high-perfor-\nmance networking, storage, and security", ". It's a breakthrough in high-perfor-\nmance networking, storage, and security. In \na large-scale data center, a single DPU can \noffload 200 CPU cores", '. In \na large-scale data center, a single DPU can \noffload 200 CPU cores. Developers can use \nthe NVIDIA DOCA SDK—the DPU equivalent \nMade in NVIDIA Omniverse']

# Page 16: ['to our CUDA SDK for GPUs—to easily create \nhigh-performance, software-defined, \ncloud-native applications and services  \nfor BlueField-2', ".\nIn September, we announced an agreement \nto acquire Arm for $40 billion to create the \nworld's premier computing company. Arm is \nthe world's most popular CPU", ". Arm is \nthe world's most popular CPU. They were \nfirst a breakout provider for the mobile \nmarket and have since expanded into new \nmarkets, including PCs, cloud, and high \nperformance computing", '. They have an \nexcellent business model', ". With NVIDIA's AI \ncapabilities, we can enhance Arm's offering \nin mobile and embedded, build a thriving \necosystem, and advance computing from the \ncloud, smartphones, PCs, self-driving cars,", 'ecosystem, and advance computing from the \ncloud, smartphones, PCs, self-driving cars, \nrobotics, and edge IoT', '. Together, we have the \nopportunity to provide greater choice in the \ndata center ecosystem', ".\nWhen we announced the acquisition, we esti-\nmated the regulatory review process would \ntake approximately 18 months, and it's \nmoving forward as expected", ". We're \nconfident that regulators will see the \nbenefits to the entire tech ecosystem", '.\nNVIDIA INVENTS  \nA COMPUTER FOR AI\nAt the beginning of the big bang of modern \nAI, we recognized the need to create a whole \nnew kind of computer for a new way of devel-\noping software', '. Software will write software \nrunning on AI computers. \nThe cornerstone of NVIDIA AI is the \nre-invention of the GPU to become Tensor \nCore GPUs', '. \nThe cornerstone of NVIDIA AI is the \nre-invention of the GPU to become Tensor \nCore GPUs. This year we surprised the \nindustry with the advancement of the NVIDIA \nAmpere architecture', '. NVIDIA A100 is a \n54-billion transistor marvel and the first \nuniversal data center GPU—a rocket engine \nfor data analytics and scientific computing, \nAI training, and inference workloads', '. What \nused to be done by two or three GPUs is \nunified within one. The versatility gives data \ncenters greater flexibility and utilization.\nTwo new inventions came with the A100', '.\nTwo new inventions came with the A100. The \nfirst is a numerical format called Tensor-\nFloat-32', '. The \nfirst is a numerical format called Tensor-\nFloat-32. It speeds up deep learning \nprocessing, so Ampere GPUs learn \nincredibly fast—20x faster than our previous \ngeneration Volta GPU', '. The second is \nMulti-Instance GPU (MIG) technology, which \ndivides a single GPU into seven smaller \ninstances', '. With processors growing larger, \nMIG provides another degree of versatility by \nletting services configure the computer with \ngiant GPUs or many smaller ones', '.\nAI needs a whole new type of computer with \nnew chips like the A100 Tensor Core GPU, \nnew system architecture, new ways to \nnetwork, new software, and new methodolo -\ngies and tools', ". We've invested billions into \nthis intuition, and it has proven helpful to \nthe industry.\nIt all comes together as DGX—a computer \nfor AI", '.\nIt all comes together as DGX—a computer \nfor AI.\nWe offer DGX as a fully integrated system, as \nwell as offer the components to the industry \nto create differentiated options', '.\nI am pleased to see so much AI research \nadvancing because of DGX—top universities, \nresearch hospitals, telcos, banks, consumer \nproducts companies, carmakers, and \naerospace companies', '. DGX helps their \nAI researchers—whose expertise is rare, \nscarce, and their work strategic. It is \nimperative to make sure they have the right \ninstrument', '. It is \nimperative to make sure they have the right \ninstrument. If software is to be written by \ncomputers, companies with the best \nsoftware engineers will need the  \nbest computers', ".\nAlong with the new Ampere GPU, we \nunveiled the new DGX, the most powerful \ncomputer we've ever made. Powered by \neight A100 GPUs, the DGX A100 replaces \nhundreds of CPU-based servers", '. Powered by \neight A100 GPUs, the DGX A100 replaces \nhundreds of CPU-based servers. The \nperformance is incredible—five petaflops  \nin a single box', '. The \nperformance is incredible—five petaflops  \nin a single box.\nAI – SOFTWARE \nTHAT WRITES SOFTWARE\nWe reinvented every layer of computing for \nAI. First, we invented Tensor Core GPUs and \nDGX', '. First, we invented Tensor Core GPUs and \nDGX. Then the suite of system software and \nacceleration libraries we call NVIDIA AI', '.\nUsing NVIDIA AI, our world-class AI experts \ndeveloped pre-trained AI frameworks that \ncustomers can customize for their applica-\ntions', '. These are skills like computer vision, \nconversational AI, recommender systems, \nAI avatars, robotics, and autonomous \nvehicles', '. These AI skills are state of the art, \ntrained to be used in production, and \noptimized for accuracy and performance.\nOne example is our Jarvis conversational AI', '. \nIt’s composed of a set of complex machine \nlearning skills—speech recognition, \nlanguage understanding, translation, and \nspeech—that lets us carry on a conversation \nwith an AI', '. Jarvis is trained on over a billion \npages of text and can recognize speech at \nworld-class 90 percent accuracy. Jarvis \nresponds interactively and can run any -\nwhere there is NVIDIA', ".\nAnother is our Merlin recommender system \nwhich learns our explicit and implicit \npreferences to suggest a new restaurant, \nmovie, or song we might like based on what \nwe've enjoyed before", '. Recommender \nsystems are the personalization engines of \nthe internet, filtering trillions of items to \npresent the ones most relevant to you. It is \nthe economic engine of internet services', '. It is \nthe economic engine of internet services.\nAnd it is one of the largest computing \ninfrastructures of most internet companies', '.\nAnd it is one of the largest computing \ninfrastructures of most internet companies. \nNVIDIA Merlin simplifies the effort to build a \nstate-of-the-art recommender system', '. \nNVIDIA Merlin simplifies the effort to build a \nstate-of-the-art recommender system.\nTHE FOUR WAVES OF AI\nAI—computers automating intelligence—is \nthe most powerful technology force of our \ntime', '. We see AI in four waves. The first was \nto reinvent computing for this new way of \ndoing software. The second came as internet \ncompanies deployed AI in the cloud', '. The second came as internet \ncompanies deployed AI in the cloud. AI \nservices recognize our speech, understand \nour meaning, and make recommendations', '. AI \nservices recognize our speech, understand \nour meaning, and make recommendations.\nThe next wave is enterprise and the industri-\nal edge', ".\nThe next wave is enterprise and the industri-\nal edge. AI will revolutionize the world's \nlargest industries—manufacturing, logistics, \nagriculture, healthcare, financial services, \ntransportation", ". Companies are integrating \n5G, AI, and autonomous machines to create \nsmart services. It's the smartphone moment \nfor the world's industries", ". It's the smartphone moment \nfor the world's industries.\nLet me give you some examples of AI at the \nindustrial edge", '.\n > Artisight uses NVIDIA AI to create an AI \nnurse assistant that continuously \nwatches over patients, converses with \npatients, and alerts nurses if it sees a \npatient has fallen', '.\n > USPS uses NVIDIA AI to recognize \nfeatures on a package as it flies by on a \nconveyor belt to sort 20 million packages \na day.']

# Page 17: ['> RecyclEye uses NVIDIA AI to recognize \nand sort items in a recycling plant.\n > Datamonsters uses NVIDIA AI to find \npackaging defects on high-velocity \nconveyor belts', '.\n > Everseen uses NVIDIA AI to do fast and \nerror-free checkout for Kroger, the \nlargest supermarket chain in the U.S', '.S.\nSomeday, the industrial edge will be trillions \nof sensors streaming continuous data to AI \nsystems that automate tasks impossible just \na few years ago', '.\nEVERYTHING THAT MOVES  \nWILL BE AUTONOMOUS\nAutonomous systems are the fourth wave. \nSelf-driving cars are iconic examples', '. \nSelf-driving cars are iconic examples.\nEverything that moves will someday be \nautonomous machines that perceive their \nenvironment, reason, plan, and act', ". Whether \nit's a robot assisting a factory worker or an \nautonomous car on the road, each has to \nmake split-second decisions to do its job \nsafely", '. The computation required is \nenormous, the software is complex, and \nsafety and security are vital. For robotics, \nour platform is NVIDIA Isaac, and for \nself-driving cars, we have NVIDIA DRIVE', '. For robotics, \nour platform is NVIDIA Isaac, and for \nself-driving cars, we have NVIDIA DRIVE.\nThe world drives 10 trillion miles a year', '.\nThe world drives 10 trillion miles a year. At a \nfew dollars per mile, transportation—mov -\ning people and things—is one of the largest \nindustries', '. DRIVE is an end-to-end AI \nsystem—from the robotic processors and \ncomputers, sensor architecture, data \nprocessing, mapping, developing the driving \nsoftware, creating the simulator and digital', 'processing, mapping, developing the driving \nsoftware, creating the simulator and digital \ntwin, fleet command operations, to road \ntesting—all of it to the highest functional', 'twin, fleet command operations, to road \ntesting—all of it to the highest functional \nsafety and cybersecurity standards', '.\nOur new system-on-a-chip NVIDIA Orin  \nwill provide the immense compute \ndemands inside the car. Orin is one of our \ngreatest design feats', ". It's based on the \nNVIDIA Ampere architecture and can  \ndeliver 254 trillion operations per second  \nto scale from driver assistance to fully \nautonomous robotaxis", '.\nMercedes-Benz, Volvo, and SAIC have \nchosen Orin for their next-generation cars, \nas have electric vehicle startups like Nio, \nXPeng, Li Auto, IM Motors, and Faraday \nFuture', '. Robotaxi leaders Cruise, Amazon \nZoox, Oxbotica, Pony', '.AI, AutoX, and Didi; and \nautonomous trucking companies TuSimple, \nVolvo, Navistar, Plus, Einride, and Locomo-\ntion have also selected NVIDIA DRIVE', ".\nThis year, we announced that Mercedes has \nchosen NVIDIA to build the world's most \nadvanced cars", ". Starting in 2024, the entire \nMercedes-Benz fleet will be powered by \nNVIDIA DRIVE—utilizing NVIDIA's end-to-\nend platform to offer autonomous driving \nsoftware and services", '.\nPowered by NVIDIA AI, robotics will  \nhelp every industry achieve new levels  \nof productivity.\nJohn Deere built agriculture machines that \n"see and spray" to reduce chemical and \nherbicide use', '.\nPostmates uses delivery robots for last-mile \nfood and grocery deliveries as the \ne-commerce trend booms', '.\nPlus One Robotics robots are used by \nFedEx to sort all kinds of packages \nat 1,300 packages per hour, helping FedEx \ndeal with the exponential increase in \ne-commerce volume', '.\nDole uses FarmWise Labs robots to remove \nweeds mechanically, eliminating the need \nfor chemical herbicides', '. The machine covers \nup to 20 acres a day and uses models trained \non over 20 million images of crops.\nAI is the engine of the fourth industrial \nrevolution', '.\nNVIDIA OMNIVERSE – VIRTUAL \nWORLDS WHERE ROBOTS LEARN\nThousands of companies are developing \nrobots that perform repetitive tasks, \ndangerous tasks, or impossible-for-human', 'robots that perform repetitive tasks, \ndangerous tasks, or impossible-for-human \ntasks and work beside people to amplify \nour productivity', '.\nTraining robots is problematic in the physical \nworld. We need a virtual world that appears \nand behaves like our actual world so that \nrobots can learn skills', '. This simulated \nvirtual world will demand the most \nadvanced graphics, physics simulation, and \nAI running on superfast computers', '.\nNVIDIA Omniverse is a virtual world \nsimulator that is physically accurate. Unlike \na game or movie, Omniverse is a real-time \nsimulation', '. Unlike \na game or movie, Omniverse is a real-time \nsimulation. And though beautiful, Omniverse \nis not art, but pure science.\nOne application of Omniverse is to create \ndigital twins', '.\nOne application of Omniverse is to create \ndigital twins. Let’s use manufacturing as an \nexample', '. Let’s use manufacturing as an \nexample. In Omniverse, we will build \nvirtual factories, train robots, and simulate \nthe entire plant in operation', '. The actual and \nvirtual factory will run in parallel. Inside \nOmniverse will be the digital twin,  \nmonitoring, tracking, and exploring ways to \nimprove factory productivity', '. Digital twins \nwill be needed for robotics, such as \nself-driving cars, surgical robots, and \nrobotic warehouses.\nOmniverse is one of our most important \ninitiatives', '.\nOmniverse is one of our most important \ninitiatives. You are going to see a lot more \nof Omniverse.\nMade in NVIDIA Omniverse']

# Page 18: ['BUILDING A UNIQUE COMPANY\nWe had an incredible year.\nI often say that we are "30 days from going \nout of business." True or not, I\'ve believed \nthis since our founding', '." True or not, I\'ve believed \nthis since our founding. Relevance and \ngrowth are not the natural state of any \ncompany—the opposite is', '. Relevance and \ngrowth are not the natural state of any \ncompany—the opposite is. So, we must work \nto stay relevant, fight the gravity of good-\nenough, and seek the extraordinary', ". So, we must work \nto stay relevant, fight the gravity of good-\nenough, and seek the extraordinary. It's a \nstruggle—easy for no one. But it is a \nstruggle we love", ".\nOur company lives by a few principles:\n > Build a company that attracts the world's \nbest people in our field, giving them a \nplace to do their life's work, where they", "best people in our field, giving them a \nplace to do their life's work, where they \ncan build a great life, and promising \nnever to squander their talents", '.\n > Do the really hard things that no one has \ndone, and in the event that we do it, will \nmove the needle for society.\n > Do things that we are uniquely built to do', '.\n > Do things that we are uniquely built to do.\n > Do things that bring us incredible joy as \nthe journey will be paved with failure \nand suffering', '.\nIf we do these things, we will build a special \ncompany that makes unique and meaningful \ncontributions to the world', '.\nWe were true to our values, we did great \nwork, and in these ways, we had an \nincredible year', '.\nI want to thank all the NVIDIA families, \nwho could work anywhere but volunteered  \nto be with us in this great adventure we \ncall NVIDIA.\nJensen Huang  \nCEO and Founder, NVIDIA \nApril 2021']

# Page 19: ['NVIDIA CORPORATION \nNOTICE OF 2021 ANNUAL MEETING \nPROXY STATEMENT AND FORM 10-K \n \n \n \n \n \n \nFORWARD -LOOKING STATEMENTS', 'FORWARD -LOOKING STATEMENTS  \nCertain statements in this document including, but not limited to, statements as to:  the impact, benefits, abilities , features , performance, and', 'availability of our products and technologies, including Ampere architecture-based GPUs, GeForce RTX GPUs, GeForce Experience, Max -Q, NVIDIA', 'DGX SuperPOD, NVIDIA Maxine, RTX, DGX, EGX, NVIDIA EGX Edge AI platform,  NVIDIA Clara, NVIDIA Bluefield DPUs, NVIDIA DOCA SDK,  Tensor', 'Core GPUs, NVIDIA A100, TensorFloat-32, MIG technology, DGX A100, NVIDIA AI, Jarvis, NVIDIA Merlin, NVIDIA Isaac, NVIDIA DRIVE, NVIDIA DRIVE', 'AC, NVIDIA DRIVE Sim, NVIDIA Omniverse, and NVIDIA Orin; t he impact of the pandemic; our contributions in combatting the pandemic and its', 'benefits and impacts; the startups using AI to revolutionize healthcare, their work and its impacts; the world b eing better prepared for the next', 'pandemic, and how our technologies will help; NVIDIA GPU Computers and Clara being the computational defense system for responding to the', 'next pandemic; COVID-19 and it being the last pandemic; accelerated computing methods impacting the pace of drug discovery; our work influencing', 'every industry in the future and shaping the world; accelerated computing being the path forward, the community it has built and its impacts; NVIDIA', 'GPU processing increases and its impacts; the success and demand for RTX; the upgrade cycle RTX is driving; the impact of the Mellanox acquisition;', 'the data center being the new unit of computing and its impacts; the benefits and impacts of the Arm acquisition and recommender systems ;', 'approval of the Arm acquisition; how software will be written; software engineers needing the best computers; the waves of AI and their impacts; AI', 'revolutionizing the world’s largest industries; what the industrial edge will be;  everything that moves being autonomous machines; the companies', 'using NVIDIA Orin; our work with Mercedes and its impact; robotics helping every industry achieve new levels of productivity; what simulated virtual', 'worlds will demand; digital twins being needed for  robotics; our company’s principles; the impact of autonomous vehicles; our growth drivers  and', 'opportunities; and our market position and strategies  are forward-looking statements that are subject to risks and uncertainties that could cause', 'results to be materially differen t than expectations', '. Important factors that could cause actual results to differ materially include: global economic', 'conditions; our reliance on third parties to manufacture, assemble, package and test our products; the impact of technological development and', 'competition; development of new products and technologies or enhancements to our existing product and technologies; market acceptance of our', "products or our partners' products; design, manufacturing or software defects; changes in consumer preferences or demands; changes in industry", 'standards and interfaces; unexpected loss of performance of our products or technologies when integrated into systems; as wel l as other factors', 'detailed from time to time in the most recent reports NVIDIA files with the Securities and Exchange Commission, or SEC, including, but not limited', 'to, its annual report on Form 10 -K and quarterly reports on Form 10 -Q', ". Copies of reports filed with the SEC are posted on the company's website \nand are available from NVIDIA without charge", '. These forward -looking statements are not guarantees of future performance and speak only as of', 'the date hereof, and, except as required by law, NVIDIA disclaims any obligation to update these forward-looking statements to reflect future events \nor circumstances', '.']

# Page 20: ['NOTICE OF 2021 ANNUAL MEETING OF STOCKHOLDERS\nDate and time: Thursday, June 3, 2021 at 11:00 a.m. Pacific Daylight Time\nLocation: Virtually at www.virtualshareholdermeeting', '.com/NVIDIA2021\nItems of business: • Election of thirteen directors nominated by the Board of Directors\n• Advisory approval of our executive compensation', '• Advisory approval of our executive compensation\n• Ratification of the selection of PricewaterhouseCoopers LLP as our independent registered \npublic accounting firm for fiscal year 2022', 'public accounting firm for fiscal year 2022\n• Approval of an amendment to our Amended and Restated Certificate of Incorporation to', '• Approval of an amendment to our Amended and Restated Certificate of Incorporation to \nincrease the number of authorized shares of common stock from 2 billion shares to 4 billion \nshares', 'increase the number of authorized shares of common stock from 2 billion shares to 4 billion \nshares\nTransaction of other business properly brought before the meeting', 'shares\nTransaction of other business properly brought before the meeting\nRecord date: You can attend and vote at the annual meeting if you were a stockholder of record at the close of', 'business on April 5, 2021', '.\nStockholder list: A list of stockholders entitled to vote at the close of business on the April 5, 2021 record date will \nbe available during the entire time of the annual meeting at www', '.virtualshareholdermeeting.com/\nNVIDIA2021 and electronically for 10 days prior to the annual meeting to registered stockholders \nfor any legally valid purpose related to the annual meeting', '.  For access to the stockholder list, \nplease contact us at shareholdermeeting@nvidia.com.\nVirtual meeting \nadmission:\nWe will be holding our annual meeting virtually at www.virtualshareholdermeeting', '.virtualshareholdermeeting.com/\nNVIDIA2021.  To participate in the annual meeting, you will need the control number included on \nyour notice of proxy materials or printed proxy card', '.\nPre-meeting \nforum:\nTo more effectively communicate with our stockholders in connection with the annual meeting, we \nhave established a pre-meeting forum located at www.proxyvote', '.proxyvote.com where you can submit \nadvance questions to us. \nYour vote is very important.  Whether or not you plan to attend the virtual annual meeting, PLEASE VOTE YOUR \nSHARES', '.  As an alternative to voting online at the virtual annual meeting, you may vote via the Internet, by telephone', 'or, if you have elected to receive a paper proxy card in the mail, by mailing the completed proxy card', '.\nImportant notice regarding the availability of proxy materials for the Annual Meeting of Stockholders to be held on \nJune 3, 2021', '.  This Notice, our Proxy Statement, our Annual Report on Form 10-K, and our Annual Review are available \nat www.nvidia.com/proxy.\nBy Order of the Board of Directors \nTimothy S', '.nvidia.com/proxy.\nBy Order of the Board of Directors \nTimothy S. Teter \nSecretary\n2788 San Tomas Expressway, Santa Clara, California 95051\nApril 23, 2021']

In [ ]:
print(final_texts)

# 출력:
# ['© 2024 NVIDIA Corporation. All rights reserved. \nNVIDIA Corporation\u2003|\u20032788 San Tomas Expressway, Santa Clara, California 95051\u2003|\u2003www.nvidia.com']

In [ ]:
# ## 3. 인덱싱 (Texts -> Embedding -> Store) + 4. 검색(Retrieval)

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENAI_API_KEY")
openai_api_key = key
os.environ['OPENAI_API_KEY'] = openai_api_key

# 파일 로드
file_path = "report2024.txt"
loader = TextLoader(file_path)
documents = loader.load()

# 텍스트를 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, separators=["]", "', '."]
)
texts = text_splitter.split_documents(documents)

# 메타데이터 추가 (페이지 번호와 문서명)
for text in texts:
    if "Page" in text.page_content:  # 페이지 번호가 명시된 줄만 처리
        page_info = text.page_content.split(":")[0]
        page_number = page_info.replace("Page ", "").strip()
        text.metadata = {
            "page_number": page_number,
            "document_name": "2024"
        }

# OpenAI 임베딩 생성
embeddings = OpenAIEmbeddings()

# Chroma 벡터스토어 저장
vectorstore = Chroma.from_documents(texts, embeddings, persist_directory="chroma_store")
vectorstore.persist()

# RAG QA 체인
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=retriever,
    return_source_documents=True,
)

# 사용자 질문 처리 함수
def query_document(query):
    # QA 체인으로 답변 생성
    result = qa_chain({"query": query})

    # 결과 처리
    sources = result["source_documents"]
    response = {
        "answer": result["result"],
        "relevant_pages": [
            {"page_number": doc.metadata.get("page_number", "Unknown"), "chunk": doc.page_content}
            for doc in sources
        ],
    }
    return response

query = "What is ERM?"
response = query_document(query)
print(json.dumps(response, indent=4, ensure_ascii=False))

# 출력:
# <ipython-input-3-9bb29170c02b>:37: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
#   embeddings = OpenAIEmbeddings()
# <ipython-input-3-9bb29170c02b>:41: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
#   vectorstore.persist()
# <ipython-input-3-9bb29170c02b>:46: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
#   llm=OpenAI(),
# <ipython-input-3-9bb29170c02b>:54: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
#   result = qa_chain({"query": query})
# {
#     "answer": " I don't know, as there is not enough context provided to determine what ERM could stand for. ",
#     "relevant_pages": [
#         {
#             "page_number": "Unknown",
#             "chunk": "', '.A"
#         },
#         {
#             "page_number": "]\n11",
#             "chunk": "]\nPage 11: ["
#         },
#         {
#             "page_number": "]\n13",
#             "chunk": "]\nPage 13: ["
#         }
#     ]
# }

In [ ]:
## 5. 생성(Generation)

from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENAI_API_KEY")
openai_api_key = key
os.environ['OPENAI_API_KEY'] = openai_api_key

llm = ChatOpenAI(temperature=0.7)

# chain 실행
response = llm.invoke("What is ERM?")

prompt = ChatPromptTemplate.from_template("You are a business expert of NVIDIA. Answer the question. <Question>: {input}")

# chain 연결
chain = prompt | llm

# chain 호출
response = chain.invoke({"input": "What is ERM?"})

print(response)

# 출력:
# content='ERM stands for Enterprise Risk Management. It is a strategic approach to identifying, assessing, and preparing for potential risks and opportunities that could affect the success of a business. ERM involves a systematic process of analyzing and managing risks across all areas of an organization, including financial, operational, strategic, and compliance risks. By implementing ERM practices, companies like NVIDIA can proactively address potential threats and capitalize on opportunities to enhance their overall performance and resilience.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 27, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-9155422b-df3d-4d6e-afb2-d85131ceec95-0'

In [ ]:
# RAG 통합 코드(3, 4, 5번 통합) - 2024
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENAI_API_KEY")
openai_api_key = key
os.environ['OPENAI_API_KEY'] = openai_api_key

# 파일 로드
file_path = "report2024.txt"
loader = TextLoader(file_path)
documents = loader.load()

# 텍스트를 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, separators=["]", "', '."]
)
texts = text_splitter.split_documents(documents)

# 메타데이터 추가 (페이지 번호와 문서명)
for text in texts:
    if "Page" in text.page_content:  # 페이지 번호가 명시된 줄만 처리
        page_info = text.page_content.split(":")[0]
        page_number = page_info.replace("Page ", "").strip()
        text.metadata = {
            "page_number": page_number,
            "document_name": "2024"
        }

# OpenAI 임베딩 생성
embeddings = OpenAIEmbeddings()

# Chroma 벡터스토어 저장
vectorstore = Chroma.from_documents(texts, embeddings, persist_directory="chroma_store")
vectorstore.persist()

# RAG QA 체인
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=retriever,
    return_source_documents=True,
)

# 사용자 질문에 대한 처리 함수
def query_document(query):
    # QA 체인을 사용하여 답변 생성
    result = qa_chain({"query": query})
    
    # 결과 처리
    sources = result["source_documents"]
    response = {
        "answer": result["result"],
        "relevant_pages": [
            {"page_number": doc.metadata.get("page_number", "Unknown"), "chunk": doc.page_content}
            for doc in sources
        ],
    }
    return response

query = ""
response_rag = query_document(query)
# print(json.dumps(response, indent=4, ensure_ascii=False))



llm = ChatOpenAI(temperature=0.7)

# chain 실행
response_gpt = llm.invoke(query)

prompt = ChatPromptTemplate.from_template("You are a business expert of NVIDIA. Answer the question. <Question>: {input}")

# chain 연결
chain = prompt | llm

# chain 호출s
response = chain.invoke({"input": query})

print(response_rag)
print()
print(response_gpt)
print()
print(response)

# # i dont know 방지 처리
# if "don't know" or "I'm sorry" or "have access" in response_gpt:
#     response = query_document(query)["answer"]

# print(response)

{'answer': '\nIt is not specified in the given context how many generative AI companies are building on NVIDIA.', 'relevant_pages': [{'page_number': 'Unknown', 'chunk': "', '.\\nNVIDIA AI for Every \\nIndustry and Enterprise\\nGenerative AI will transform every \\nindustry', '. NVIDIA has already applied \\nAI to build several multi-billion-\\ndollar verticals—gaming, healthcare, \\nautomotive, and robotics', '. We are \\nnow bringing NVIDIA’s accelerated \\ncomputing and AI to the enterprise. \\nEnterprises have not historically been a \\nmarket for GPUs and high-performance \\ncomputing. Generative AI changes that"}, {'page_number': 'Unknown', 'chunk': "', '.\\nNVIDIA AI for Every \\nIndustry and Enterprise\\nGenerative AI will transform every \\nindustry', '. NVIDIA has already applied \\nAI to build several multi-billion-\\ndollar verticals—gaming, healthcare, \\nautomotive, and robotics', '. We are \\nnow bringing NVIDIA’s accelerated \\ncomputing and AI to the enterprise. \\nEnte

In [ ]:
# RAG 통합 코드 - 2023
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENAI_API_KEY")
openai_api_key = key
os.environ['OPENAI_API_KEY'] = openai_api_key

# 파일 로드
file_path = "report2023.txt"
# file_path="C:\\Users\\Admin\\Project1\\report2024.txt"
loader = TextLoader(file_path)
documents = loader.load()

# 텍스트를 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, separators=["]", "', '."]
)
texts = text_splitter.split_documents(documents)

# 메타데이터 추가 (페이지 번호와 문서명)
for text in texts:
    if "Page" in text.page_content:  # 페이지 번호가 명시된 줄만 처리
        page_info = text.page_content.split(":")[0]
        page_number = page_info.replace("Page ", "").strip()
        text.metadata = {
            "page_number": page_number,
            "document_name": "2023"
        }

# OpenAI 임베딩 생성
embeddings = OpenAIEmbeddings()

# Chroma 벡터스토어 저장
vectorstore = Chroma.from_documents(texts, embeddings, persist_directory="chroma_store")
vectorstore.persist()

# RAG QA 체인
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=retriever,
    return_source_documents=True,
)

# 사용자 질문에 대한 처리 함수
def query_document(query):
    # QA 체인을 사용하여 답변 생성
    result = qa_chain({"query": query})
    
    # 결과 처리
    sources = result["source_documents"]
    response = {
        "answer": result["result"],
        "relevant_pages": [
            {"page_number": doc.metadata.get("page_number", "Unknown"), "chunk": doc.page_content}
            for doc in sources
        ],
    }
    return response

query = "During fiscal year 2024, how much NVIDIA paid in quarterly cash dividends?"
response_rag = query_document(query)
# print(json.dumps(response, indent=4, ensure_ascii=False))



llm = ChatOpenAI(temperature=0.7)

# chain 실행
response_gpt = llm.invoke(query)

prompt = ChatPromptTemplate.from_template("You are a business expert of NVIDIA. Answer the question. <Question>: {input}")

# chain 연결
chain = prompt | llm

# chain 호출s
response = chain.invoke({"input": query})

print(response_rag)
print()
print(response_gpt)
print()
print(response)

# i dont know 방지 처리
if "don't know" or "I'm sorry" or "have access" in response_gpt:
    response = query_document(query)["answer"]

print(response)

content="I'm sorry, but as an AI language model, I do not have access to real-time or specific financial data. I recommend checking NVIDIA's official financial reports or website for the most up-to-date information on their quarterly cash dividends during fiscal year 2024." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 38, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-a14d0731-8392-4be8-b1e2-a6388056f85a-0'


In [ ]:
# RAG 통합 코드 - 2022
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENAI_API_KEY")
openai_api_key = key
os.environ['OPENAI_API_KEY'] = openai_api_key

# 파일 로드
file_path = "report2022.txt"
# file_path="C:\\Users\\Admin\\Project1\\report2024.txt"
loader = TextLoader(file_path)
documents = loader.load()

# 텍스트를 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, separators=["]", "', '."]
)
texts = text_splitter.split_documents(documents)

# 메타데이터 추가 (페이지 번호와 문서명)
for text in texts:
    if "Page" in text.page_content:  # 페이지 번호가 명시된 줄만 처리
        page_info = text.page_content.split(":")[0]
        page_number = page_info.replace("Page ", "").strip()
        text.metadata = {
            "page_number": page_number,
            "document_name": "2022"
        }

# OpenAI 임베딩 생성
embeddings = OpenAIEmbeddings()

# Chroma 벡터스토어 저장
vectorstore = Chroma.from_documents(texts, embeddings, persist_directory="chroma_store")
vectorstore.persist()

# RAG QA 체인
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=retriever,
    return_source_documents=True,
)

# 사용자 질문에 대한 처리 함수
def query_document(query):
    # QA 체인을 사용하여 답변 생성
    result = qa_chain({"query": query})
    
    # 결과 처리
    sources = result["source_documents"]
    response = {
        "answer": result["result"],
        "relevant_pages": [
            {"page_number": doc.metadata.get("page_number", "Unknown"), "chunk": doc.page_content}
            for doc in sources
        ],
    }
    return response

query = "During fiscal year 2024, how much NVIDIA paid in quarterly cash dividends?"
response_rag = query_document(query)
# print(json.dumps(response, indent=4, ensure_ascii=False))



llm = ChatOpenAI(temperature=0.7)

# chain 실행
response_gpt = llm.invoke(query)

prompt = ChatPromptTemplate.from_template("You are a business expert of NVIDIA. Answer the question. <Question>: {input}")

# chain 연결
chain = prompt | llm

# chain 호출s
response = chain.invoke({"input": query})

print(response_rag)
print()
print(response_gpt)
print()
print(response)

# i dont know 방지 처리
if "don't know" or "I'm sorry" or "have access" in response_gpt:
    response = query_document(query)["answer"]

print(response)

In [ ]:
# RAG 통합 코드 - 2024
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENAI_API_KEY")
openai_api_key = key
os.environ['OPENAI_API_KEY'] = openai_api_key

# 파일 로드
file_path = "report2021.txt"
# file_path="C:\\Users\\Admin\\Project1\\report2024.txt"
loader = TextLoader(file_path)
documents = loader.load()

# 텍스트를 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, separators=["]", "', '."]
)
texts = text_splitter.split_documents(documents)

# 메타데이터 추가 (페이지 번호와 문서명)
for text in texts:
    if "Page" in text.page_content:  # 페이지 번호가 명시된 줄만 처리
        page_info = text.page_content.split(":")[0]
        page_number = page_info.replace("Page ", "").strip()
        text.metadata = {
            "page_number": page_number,
            "document_name": "2021"
        }

# OpenAI 임베딩 생성
embeddings = OpenAIEmbeddings()

# Chroma 벡터스토어 저장
vectorstore = Chroma.from_documents(texts, embeddings, persist_directory="chroma_store")
vectorstore.persist()

# RAG QA 체인
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=retriever,
    return_source_documents=True,
)

# 사용자 질문에 대한 처리 함수
def query_document(query):
    # QA 체인을 사용하여 답변 생성
    result = qa_chain({"query": query})
    
    # 결과 처리
    sources = result["source_documents"]
    response = {
        "answer": result["result"],
        "relevant_pages": [
            {"page_number": doc.metadata.get("page_number", "Unknown"), "chunk": doc.page_content}
            for doc in sources
        ],
    }
    return response

query = "During fiscal year 2024, how much NVIDIA paid in quarterly cash dividends?"
response_rag = query_document(query)
# print(json.dumps(response, indent=4, ensure_ascii=False))



llm = ChatOpenAI(temperature=0.7)

# chain 실행
response_gpt = llm.invoke(query)

prompt = ChatPromptTemplate.from_template("You are a business expert of NVIDIA. Answer the question. <Question>: {input}")

# chain 연결
chain = prompt | llm

# chain 호출s
response = chain.invoke({"input": query})

print(response_rag)
print()
print(response_gpt)
print()
print(response)

# i dont know 방지 처리
if "don't know" or "I'm sorry" or "have access" in response_gpt:
    response = query_document(query)["answer"]

print(response)